# Análise de sentimento - Comentários de produtos eletrônicos do youtube - Armazenamento no Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from datetime import datetime
import pandas as pd
import time
import numpy as np

cred = credentials.Certificate('node/serviceAccountKey_teste.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [2]:
comments_ref = db.collection(u'video-comments-count').document('count')
docs = comments_ref.get()

In [3]:
#print(f'Document data: {str(docs.to_dict())}')

#print(docs.to_dict()['video-comments-count'])
arr_ids = [] 
arr_counts = []

for comment in docs.to_dict()['video-comments-count'].split("|"):
    arr_ids.append(str(comment.split(",")[0]))
    arr_counts.append(int(comment.split(",")[1]))

dict_to_dataframe = {
    "comments_id": arr_ids,
    "comments_count": arr_counts
}

df = pd.DataFrame(dict_to_dataframe)

In [4]:
df = df.loc[df['comments_count'] != 100000]

In [5]:
video_comments_reviews = pd.DataFrame()
for cid in df['comments_id']:

    try:
        collections = db.collection('video-comments-reviews').document(cid).collections()
        for collection in collections:
            for doc in collection.stream():
                review = doc.to_dict()
                review['comments_id'] = cid
                video_comments_reviews = video_comments_reviews.append(review, ignore_index=True)

    except:
        print(cid,' Não encontrado')

In [6]:
video_comments_reviews

,comments_id,reviews-count,type,updatedAt
0,0.UgwCN3LYe6_7mhQwvPF4AaABAg,99997,-1.0,2021-08-05 03:00:00+00:00
1,0.UgwCN3LYe6_7mhQwvPF4AaABAg,99997,-1.0,2021-08-05 03:00:00+00:00
2,0.UgwCN3LYe6_7mhQwvPF4AaABAg,99997,0.0,2021-08-05 03:00:00+00:00
3,0.UgwCN3LYe6_7mhQwvPF4AaABAg,99997,1.0,2021-08-05 03:00:00+00:00
4,1.Ugy7sSdqZBNru0HSeb94AaABAg,99996,-1.0,2021-07-15 02:33:42.314000+00:00
5,10.UgyfC08zO3XB_3BBc_l4AaABAg,99997,1.0,2021-08-05 03:00:00+00:00
6,10.UgyfC08zO3XB_3BBc_l4AaABAg,99997,1.0,2021-08-05 03:00:00+00:00
7,100.UgxWku9bqCY816wsCCp4AaABAg,99996,-1.0,2021-08-05 03:00:00+00:00
8,100.UgxWku9bqCY816wsCCp4AaABAg,99997,0.0,2021-07-15 02:33:18.676000+00:00


In [7]:
video_comments = pd.read_json('node/video_comments.json', convert_axes=True)
video_comments = video_comments.T.reset_index().rename(columns={'index': 'comments_id'})

In [8]:
video_comments.head(2)

,comments_id,comment-date,comment-author,comment,comment-likes,video-id,video-title,video-date,video-link,video-likes,video-views,video-channel,video-comment-count
0,0.UgwozCGH-HDyIES2ciF4AaABAg,2020-12-20T22:44:18Z,POP 0stras,Uma porcaria. Comprei faz 3 meses. Muito arrep...,0,jSv8fxe3_KU,"GALAXY NOTE 10 LITE, FAZ SENTIDO COMPRAR? [Han...",2020-02-19T16:00:09Z,https://www.youtube.com/watch?v=jSv8fxe3_KU,18773,307422,Canaltech,1013
1,1.Ugy7sSdqZBNru0HSeb94AaABAg,2019-08-07T17:19:55Z,Caio Ellery,esse note 10+ vai custar uma *facada*,0,flYuZ1KBaKM,É ASSIM O SAMSUNG GALAXY NOTE 10! SAMSUNG ERRO...,2019-08-06T22:02:24Z,https://www.youtube.com/watch?v=flYuZ1KBaKM,10581,128218,Dudu Rocha,714


In [9]:
video_comments_reviews["final_type"] = np.nan
temp_type = []

for comment in video_comments_reviews['comments_id'].unique():
    types = video_comments_reviews['type'].loc[video_comments_reviews['comments_id'] == comment]
    
    counts = types.value_counts(ascending=False).reset_index()
    mx = counts['type'].max()
    same_values = -1

    if types.count() != 1:

        for i,ty in enumerate(counts['index']):
            if mx == counts['type'].iloc[i]:
                same_values = same_values+1

        if same_values != 0:
            for idx,types in enumerate(video_comments_reviews["final_type"].loc[video_comments_reviews['comments_id'] == comment]):
                temp_type.append(0)
        else:
            for idx,types in enumerate(video_comments_reviews["final_type"].loc[video_comments_reviews['comments_id'] == comment]):
                temp_type.append(counts['index'].loc[counts['type'] == mx][0])

    else:
        temp_type.append(mx)


video_comments_reviews["final_type"] = temp_type

video_comments_reviews

,comments_id,reviews-count,type,updatedAt,final_type
0,0.UgwozCGH-HDyIES2ciF4AaABAg,99999.0,-1.0,2021-07-18 16:23:30.616000+00:00,-1.0
1,0.UgwozCGH-HDyIES2ciF4AaABAg,99998.0,-1.0,2021-07-21 15:58:29.163000+00:00,-1.0
2,3.Ugzuo_fNCeU10B0Iqv54AaABAg,99999.0,-1.0,2021-07-15 02:41:05.355000+00:00,1.0
3,3.Ugzuo_fNCeU10B0Iqv54AaABAg,99997.0,1.0,2021-07-23 20:24:53.879000+00:00,1.0
4,3.Ugzuo_fNCeU10B0Iqv54AaABAg,99998.0,1.0,2021-07-15 21:55:14.446000+00:00,1.0
...,...,...,...,...,...
7782,9995.UgzCH5fbhBklPp22nbd4AaABAg,99998.0,1.0,2021-07-17 19:34:39.862000+00:00,1.0
7783,9997.Ugwu1TjlQHr-soAyagt4AaABAg,99999.0,0.0,2021-07-27 16:57:04.007000+00:00,0.0
7784,9997.Ugwu1TjlQHr-soAyagt4AaABAg,99999.0,0.0,2021-07-18 16:54:51.949000+00:00,0.0
7785,9999.UgwIP04x6RP8Op4bg3R4AaABAg,99999.0,0.0,2021-07-16 21:16:37.958000+00:00,0.0


In [10]:
video_comments_reviews_final_type = video_comments_reviews.drop(columns=['type','updatedAt','reviews-count']).drop_duplicates(subset=None, keep='first')

In [19]:
video_comments_reviews_final_type.count()

comments_id    4833
final_type     4833
dtype: int64

In [12]:
video_comments = pd.merge(video_comments, video_comments_reviews_final_type, on="comments_id")


In [13]:
pd.DataFrame(video_comments).to_csv("video_comments_final_types.csv", index=False)

In [14]:
video_comments

,comments_id,comment-date,comment-author,comment,comment-likes,video-id,video-title,video-date,video-link,video-likes,video-views,video-channel,video-comment-count,final_type
0,0.UgwozCGH-HDyIES2ciF4AaABAg,2020-12-20T22:44:18Z,POP 0stras,Uma porcaria. Comprei faz 3 meses. Muito arrep...,0,jSv8fxe3_KU,"GALAXY NOTE 10 LITE, FAZ SENTIDO COMPRAR? [Han...",2020-02-19T16:00:09Z,https://www.youtube.com/watch?v=jSv8fxe3_KU,18773,307422,Canaltech,1013,-1.0
1,3.Ugzuo_fNCeU10B0Iqv54AaABAg,2021-04-19T01:28:56Z,tout a commencé en France,Assistindo com o meu a51 e agora estou querend...,10,6LwYbm71AJM,Review GALAXY S21 ULTRA! Agora sim a SAMSUNG f...,2021-04-07T22:00:01Z,https://www.youtube.com/watch?v=6LwYbm71AJM,5287,91701,Be!Tech,430,1.0
2,4.UgzaPxJpaltmYRPm7WZ4AaABAg,2021-06-03T22:58:59Z,Tasio de Luanda Luanda,Bom eu tenho problemas e com poeira água e queda,0,5JWFHfF4nGg,SAMSUNG GALAXY M31 // um INTERMEDIÁRIO com 600...,2020-07-09T21:19:36Z,https://www.youtube.com/watch?v=5JWFHfF4nGg,22040,304881,EscolhaSegura,1413,1.0
3,5.Ugxg7N1mhRETPeDd0jB4AaABAg,2021-03-04T02:37:38Z,ᴘ ʀ ɪ ɴ ᴄ ᴇ s ᴀッ™,Se no Samsung vier os emoji do iPhone eu me re...,0,vOVz5rPk2vU,GALAXY NOTE 10 e GALAXY NOTE 10 PLUS!,2019-08-07T23:00:51Z,https://www.youtube.com/watch?v=vOVz5rPk2vU,196838,2130935,Coisa de Nerd,5604,0.0
4,6.Ugwxvye3XUXp-6GdwxV4AaABAg,2021-03-03T20:09:55Z,Eu Mesmo,"Smartphone gamer muito excelentente msm, mas f...",0,TdALKXTVZRU,ROG PHONE 3: UNBOXING DO SMARTPHONE MAIS PODER...,2020-07-22T15:30:08Z,https://www.youtube.com/watch?v=TdALKXTVZRU,62254,942191,Loop Infinito,2266,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4827,9990.Ugz4MHAKLSZjySMHC8p4AaABAg,2020-10-22T19:19:10Z,Saulo Lima,"Entrada em cima , n gosto ! Gosto de entrada e...",0,uwlW-aTc5j4,MOTOROLA EDGE+! O MELHOR MOTOROLA JA FEITO! Ma...,2020-07-02T11:59:43Z,https://www.youtube.com/watch?v=uwlW-aTc5j4,11290,117110,Dudu Rocha,835,1.0
4828,9993.Ugy9hH2G4Nq55fBuJ414AaABAg,2020-12-06T20:06:26Z,Andressa Tenório,Sempre quis dizer isso.... assistindo do meu a...,1,w8ZokAmv8yE,Review Samsung Galaxy A71: um celular intermed...,2020-08-10T17:25:44Z,https://www.youtube.com/watch?v=w8ZokAmv8yE,15444,257622,TecMundo,1155,1.0
4829,9995.UgzCH5fbhBklPp22nbd4AaABAg,2021-06-29T03:21:41Z,XTRAJ4DO FF 🇧🇷,To doido pra ganhar o meu moto g9 play,0,kX2xqUDqfRA,Motorola Moto G9 Play vs Moto G9 Power - QUAL ...,2021-02-04T14:51:00Z,https://www.youtube.com/watch?v=kX2xqUDqfRA,20068,290832,Gesiel Taveira,1108,1.0
4830,9997.Ugwu1TjlQHr-soAyagt4AaABAg,2020-12-27T22:47:16Z,Gu,Mi 10t pro ou mi 10 pro?,0,MSqN27wR-gI,MI 10T PRO: TOP de LINHA XIAOMI com PREÇO de I...,2020-12-27T22:20:22Z,https://www.youtube.com/watch?v=MSqN27wR-gI,2286,65595,TudoCelular,430,0.0


In [20]:
video_comments_reviews_final_type.count()

comments_id    4833
final_type     4833
dtype: int64